## Setup

In [1]:
import firebase_admin
from firebase_admin import credentials, firestore, auth
import pandas as pd


SERVICE_ACCOUNT_KEY = "./../serviceAccountKey.json"
FILE_PATH = "./../Documentation\SETS Durbar 2025 - Secretaries Bio Data (Responses).xlsx"

In [2]:
cred = credentials.Certificate(SERVICE_ACCOUNT_KEY)
firebase_admin.initialize_app(cred)
db = firestore.client()

In [5]:
df = pd.read_excel(FILE_PATH)

## Preprocess

In [6]:
df.columns = df.columns.str.lower().str.strip().str.replace(' ', '_').str.replace("'", "")
df.rename(columns={
    "if_self-employed_:_business_name_/_if_salaried_:_company_name": "company_name",
    "role_in_business_/_designation_in_company": "designation",
    "full_name": "name"}, inplace=True)
df.drop(columns=['timestamp'], inplace=True)

In [8]:
df["name_lower"] = df["name"].str.lower()
df["company_name_lower"] = df["company_name"].str.lower()
df["club_name_lower"] = df["club_name"].str.lower()
df.fillna("", inplace=True)

In [9]:
df.columns

Index(['email_address', 'name', 'date_of_birth', 'sex', 'photograph',
       'spouses_name', 'wedding_anniversary', 'phone', 'email', 'club_name',
       'rotarian_since', 'rotary_foundation_title', 'residential_address',
       'company_name', 'designation', 'type_of_business', 'business_address',
       'business_website', 'about_your_business', 'emergency_contact_name',
       'emergency_contact_relationship', 'emergency_contact_phone',
       't_shirt_size', 'shirt_size', 'meal_preference', 'name_lower',
       'company_name_lower', 'club_name_lower'],
      dtype='object')

In [10]:
data = df.to_dict(orient='records')

## Upsert Users

- Check for email existence in firebase auth, if it exists then update record in firestore
- If email does not exist then create new user in firebase auth and then create new record in firestore with doc id = user.uid

In [11]:
for user_data in data:
    email = user_data.get("email")
    if not email:
        print("Skipping record without email:", user_data)
        continue
    
    try:
        # Check if user exists in Firebase Auth
        user = auth.get_user_by_email(email)
        print(f"User exists: {email} (UID: {user.uid}). Updating record")
        
        # Update Firestore record
        doc_ref = db.collection("users").document(user.uid)
        doc_ref.set(user_data | {"role": "member", "support": False}, merge=True)  # `merge=True` updates the document
        
    except auth.UserNotFoundError:
        print(f"User does not exist: {email}")
        
        # Create new user in Firebase Auth
        new_user = auth.create_user(email=email, password=email.split('@')[0] + '_password')
        print(f"Created new user: {email} (UID: {new_user.uid})")
        
        # Create new Firestore record
        doc_ref = db.collection("users").document(new_user.uid)
        doc_ref.set(user_data | {"role": "member", "support": False})

User exists: deepan_90@live.com (UID: pjgqrIvzbhXBAHx0WM1qxT4M1va2). Updating record
User exists: rrssbl@gmail.com (UID: Itl7GUp6bQW5jOuGVfHq2op9col2). Updating record
User exists: arunmahadevan@hotmail.com (UID: wYCW25rYsbRqtANt7gAc7vjqNxh2). Updating record
User exists: ravi.padmanaban@gmail.com (UID: Dz2YEaWJx5Xth7dfJVkJV9AqBWs2). Updating record
User exists: sanjusuresh@mgrjanaki.ac.in (UID: 48ywCUWkqnP7IwVM3StBztjZ1Ar1). Updating record
User exists: umamahes03@gmail.com (UID: o7TUgcOfifPfHHBAOqlGfyxXXyk2). Updating record
User exists: vichandrasekar@gmail.com (UID: Ln4aDPzY4NatD8dx6wwXqysN7xG3). Updating record
User exists: mv.karthick@yahoo.com (UID: f4Xuy2P6qBbKnMnwEVg3sPLv28i2). Updating record
User exists: indira.gautam@gmail.com (UID: H1OEHEHxMzdIbhmMOjUdUYJ6cRr2). Updating record
User exists: ganeshsivam76@gmail.com (UID: OycsL7BkHcgMhrcxGzoGDJ6Nvk53). Updating record
User exists: shoba.rebecca@gmail.com (UID: oknv5vP2wAZKVO94zxHoKD1ZYSJ3). Updating record
User exists: thiru